# IMPORT

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf #tensorflow
import numpy as np #numpy > save loss .. 
from collections import OrderedDict #layer ..
import os, random #dir, random..
import pickle #save & load
import math #xavier..
%matplotlib inline
from matplotlib import pyplot as plt

print(tf.__version__)

# PATH

In [ ]:
#path
#DATA_PATH = './cifar10/cifar-10-batches-py/' #python pickle
DATA_PATH = './cifar10/cifar-10-batches-bin/' #binary
CHECKPOINT = 'simple_res_net_0606.ckpt'
SAVE_PATH = './checkpoint/res_0606/'
BOARD_PATH = './tensorboard/board_res_0606/'
TEST_FILE_PATH='./cifar10/cifar-10-batches-py/test_batch'

# PARAMETERS

In [ ]:
#parameters

INPUT_SIDE = 32
INPUT_SIZE = INPUT_SIDE * INPUT_SIDE
N_CHANNEL = 3
BATCH_SIZE = 64
EPOCHS = 1500
LR = 0.1
LR_DECAY_RATE = 0.5

LABEL_BYTES = 1 
NUM_EXAMPLES_PER_EPOCH = 50000 #NUMBER OF TRAIN DATA SET
NUM_BATCHES_PER_EPOCH = NUM_EXAMPLES_PER_EPOCH / BATCH_SIZE
NUM_TESTSET_PER_EPOCH = 10000
NUM_TEST_BATCHES_PER_EPOCH = NUM_TESTSET_PER_EPOCH / BATCH_SIZE

LIST_CLASS=['airplane', 'automobile', 'birds', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
N_CLASSES = len(LIST_CLASS)

#loss & accuracy save
TRAIN_OUT_FILE_NAME = 'RES_NET_CIFAR10_0606.log'
train_loss_out = open('TRAIN_LOSS'+TRAIN_OUT_FILE_NAME, 'w')
train_accr_out = open('TRAIN_ACCURACY'+TRAIN_OUT_FILE_NAME, 'w')

TEST_OUT_FILE_NAME = 'RES_NET_CIFAR10_0606.log'
test_loss_out = open('TEST_LOSS'+TEST_OUT_FILE_NAME, 'w')
test_accr_out = open('TEST_ACCURACY'+TEST_OUT_FILE_NAME, 'w')

print("input image size : {}".format(INPUT_SIZE))
print("image channel : {}".format(N_CHANNEL))
print("batch size : {}".format(BATCH_SIZE))
print("num of class : {}".format(N_CLASSES))
print("training epochs : {}".format(EPOCHS))
print("learning rate : {}".format(LR))
print("learning decay rate : {}".format(LR_DECAY_RATE))

# FUNCTIONS

In [ ]:
#xavier initialization
def xavier_init(n_inputs, n_outputs, uniform=True):
    """Set the parameter initialization using the method described.
    This method is designed to keep the scale of the gradients roughly the same
    in all layers.
    Xavier Glorot and Yoshua Bengio (2010):
           Understanding the difficulty of training deep feedforward neural
           networks. International conference on artificial intelligence and
           statistics.
    Args:
    n_inputs: The number of input nodes into each output.
    n_outputs: The number of output nodes for each input.
    uniform: If true use a uniform distribution, otherwise use a normal.
    Returns:
    An initializer.
    """
    if uniform:
        # 6 was used in the paper.
        init_range = math.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        # 3 gives us approximately the same limits as above since this repicks
        # values greater than 2 standard deviations from the mean.
        stddev = math.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

#CIFAR 데이터 읽어오기 
def read_dataset(filename_queue):
    class DataRecord(object):
        pass
    
    result = DataRecord()
    
    label_bytes = LABEL_BYTES
    result.height = INPUT_SIDE
    result.width = INPUT_SIDE
    result.depth = N_CHANNEL
    image_bytes = result.height * result.width * result.depth
    record_bytes = label_bytes + image_bytes
    
    reader = tf.FixedLengthRecordReader(record_bytes = record_bytes)
    result.key, value = reader.read(filename_queue)
    print (value)
    record_bytes = tf.decode_raw(value, tf.uint8)
    
    result.label = tf.cast(tf.strided_slice(record_bytes, [0], [label_bytes]), tf.int32) #input, begin, end로 잘라냄 > 0~1
    
    depth_major = tf.reshape(tf.strided_slice(record_bytes, [label_bytes], # 1부터 image_bytes까지 image를 잘라낸 후 
                                              [label_bytes + image_bytes]), #channel x height x width로 변환함.
                             [result.depth, result.height, result.width])
    
    result.uint8image = tf.transpose(depth_major, [1,2,0]) #channel x height x width >>> height x width x chennl로 변경
    
    return result

#label, image 세팅해줌 
def _generate_image_and_label_batch(image, label, min_queue_examples,
                                    batch_size, shuffle):
    """Construct a queued batch of images and labels.
    Args:
      image: 3-D Tensor of [height, width, 3] of type.float32.
      label: 1-D Tensor of type.int32
      min_queue_examples: int32, minimum number of samples to retain
        in the queue that provides of batches of examples.
      batch_size: Number of images per batch.
      shuffle: boolean indicating whether to use a shuffling queue.
    Returns:
      images: Images. 4D tensor of [batch_size, height, width, 3] size.
      labels: Labels. 1D tensor of [batch_size] size.
    """
    # Create a queue that shuffles the examples, and then
    # read 'batch_size' images + labels from the example queue.
    num_preprocess_threads = 16 #프로세스 스레드 
    if shuffle: #셔플 하면 
        images, label_batch = tf.train.shuffle_batch([image, label],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples + 3 * batch_size,
            min_after_dequeue=min_queue_examples) #이거만 해주면 batch와 동일함. 
    #Minimum number elements in the queue after a dequeue, used to ensure a level of mixing of elements.
    else:
        images, label_batch = tf.train.batch([image, label],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples + 3 * batch_size)

    # Display the training images in the visualizer.
    tf.summary.image('images', images) #걍 보드용인듯

    return images, tf.reshape(label_batch, [batch_size])


def distorted_inputs(data_dir, batch_size): #좀 드럽게 인풋받아오기 
    """Construct distorted input for CIFAR training using the Reader ops.
    Args:
      data_dir: Path to the CIFAR-10 data directory.
      batch_size: Number of images per batch.
    Returns:
      images: Images. 4D tensor of [batch_size, IMAGE_SIZE, IMAGE_SIZE, 3] size.
      labels: Labels. 1D tensor of [batch_size] size.
    """
    filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i) for i in range(1, 6)] 
    for f in filenames:
        if not tf.gfile.Exists(f):
            raise ValueError('Failed to find file: ' + f)

    # Create a queue that produces the filenames to read.
    filename_queue = tf.train.string_input_producer(filenames)

    # Read examples from files in the filename queue.
    read_input = read_dataset(filename_queue)
    reshaped_image = tf.cast(read_input.uint8image, tf.float32)

    height = INPUT_SIDE
    width = INPUT_SIDE

    # Image processing for training the network.  Note the many random
    # distortions applied to the image.

    # Randomly crop a [height, width] section of the image.
    distorted_image = tf.random_crop(reshaped_image, [height, width, 3]) # 랜덤으로 이미지를 크롭 h x w x c

    # Randomly flip the image horizontally.
    distorted_image = tf.image.random_flip_left_right(distorted_image) # 또 랜덤으로 좌우 플립 random_flip_up_down도 있긴 함
    #이런 느낌..? 으로 맥스텔타가지고 유니폼으로 어쩌구저쩌구
    #delta = random_ops.random_uniform([], -max_delta, max_delta, seed=seed)
    #return adjust_brightness(image, delta)

    # Because these operations are not commutative, consider randomizing
    # the order their operation.
    distorted_image = tf.image.random_brightness(distorted_image,    #랜덤으로 밝기 조절 
                                               max_delta=63)
    distorted_image = tf.image.random_contrast(distorted_image,      #콘트라스트 조절 랜덤. 기존 픽셀값의 최소 0.2 최대 1.8
                                             lower=0.2, upper=1.8)

    # Subtract off the mean and divide by the variance of the pixels.
    float_image = tf.image.per_image_standardization(distorted_image) #표준화시킴. mean으로빼고 var로 나눔

    # Set the shapes of tensors.
    float_image.set_shape([height, width, 3])
    read_input.label.set_shape([1])

    # Ensure that the random shuffling has good mixing properties.
    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH * min_fraction_of_examples_in_queue)
    print('Filling queue with %d CIFAR images before starting to train. '
         'This will take a few minutes.' % min_queue_examples)

    # Generate a batch of images and labels by building up a queue of examples.
    return _generate_image_and_label_batch(float_image, read_input.label,
                                         min_queue_examples, batch_size,
                                         shuffle=True)


def inputs(eval_data, data_dir, batch_size):
    """Construct input for CIFAR evaluation using the Reader ops.
    Args:
    eval_data: bool, indicating if one should use the train or eval data set.
    data_dir: Path to the CIFAR-10 data directory.
    batch_size: Number of images per batch.
    Returns:
    images: Images. 4D tensor of [batch_size, IMAGE_SIZE, IMAGE_SIZE, 3] size.
    labels: Labels. 1D tensor of [batch_size] size.
    """
    if not eval_data:
        filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i)
                     for i in xrange(1, 6)]
        num_examples_per_epoch = NUM_EXAMPLES_PER_EPOCH
    else:
        filenames = [os.path.join(data_dir, 'test_batch.bin')]
        num_examples_per_epoch = NUM_TESTSET_PER_EPOCH

    for f in filenames:
        if not tf.gfile.Exists(f):
             raise ValueError('Failed to find file: ' + f)

    # Create a queue that produces the filenames to read.
    filename_queue = tf.train.string_input_producer(filenames)

    # Read examples from files in the filename queue.
    read_input = read_dataset(filename_queue)
    reshaped_image = tf.cast(read_input.uint8image, tf.float32)

    height = INPUT_SIDE
    width = INPUT_SIDE

    # Image processing for evaluation.
    # Crop the central [height, width] of the image.
    resized_image = tf.image.resize_image_with_crop_or_pad(reshaped_image,
                                                         height, width)

    # Subtract off the mean and divide by the variance of the pixels.
    float_image = tf.image.per_image_standardization(resized_image)

    # Set the shapes of tensors.
    float_image.set_shape([height, width, 3])
    read_input.label.set_shape([1])

    # Ensure that the random shuffling has good mixing properties.
    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH * min_fraction_of_examples_in_queue)

    # Generate a batch of images and labels by building up a queue of examples.
    return _generate_image_and_label_batch(float_image, read_input.label,
                                         min_queue_examples, batch_size,
                                         shuffle=False)


print("FUNCTIONS READY")

# NETWORK PARAMETERS

In [ ]:
#NETWORK PARAMETERS

stddev = 0.1

weights = {
    #'conv' : tf.Variable(tf.random_normal([3, 3, N_CHANNEL, 16], stddev=stddev), name='conv'),
    'conv' : tf.get_variable(name="conv", shape=[3, 3, N_CHANNEL, 16], initializer=xavier_init(N_CHANNEL, 16)),
    'conv1_1x1' : tf.get_variable(name="conv1_1x1", shape=[1, 1, 16, 4], initializer=xavier_init(16, 4)),
    'conv1_3x3' : tf.get_variable(name="conv1_3x3", shape=[3, 3, 4, 4], initializer=xavier_init(4, 4)),
    'conv1_1x1_16' : tf.get_variable(name="conv1_1x1_16", shape=[1, 1, 4, 16], initializer=xavier_init(4, 16)),
    'conv2_1x1' : tf.get_variable(name="conv2_1x1", shape=[1, 1, 16, 4], initializer=xavier_init(16, 4)),
    'conv2_3x3' : tf.get_variable(name="conv2_3x3", shape=[3, 3, 4, 4], initializer=xavier_init(4, 4)),
    'conv2_1x1_16' : tf.get_variable(name="conv2_1x1_16", shape=[1, 1, 4, 16], initializer=xavier_init(4, 16)),
    
    # conv 16 + conv2 16 = 32filters 
    
    'conv3_1x1' : tf.get_variable(name="conv3_1x1", shape=[1, 1, 32, 8], initializer=xavier_init(32, 8)),
    'conv3_3x3' : tf.get_variable(name="conv3_3x3", shape=[3, 3, 8, 8], initializer=xavier_init(8, 8)),
    'conv3_1x1_32' : tf.get_variable(name="conv3_1x1_32", shape=[1, 1, 8, 32], initializer=xavier_init(8, 32)),
    'conv4_1x1' : tf.get_variable(name="conv4_1x1", shape=[1, 1, 32, 8], initializer=xavier_init(32, 8)),
    'conv4_3x3' : tf.get_variable(name="conv4_3x3", shape=[3, 3, 8, 8], initializer=xavier_init(8, 8)),
    'conv4_1x1_32' : tf.get_variable(name="conv4_1x1_32", shape=[1, 1, 8, 32], initializer=xavier_init(8, 32)),
    
    # conv2 지난거32 + conv4 32 = 64
    
    'conv5_1x1' : tf.get_variable(name="conv5_1x1", shape=[1, 1, 64, 16], initializer=xavier_init(64, 16)),
    'conv5_3x3' : tf.get_variable(name="conv5_3x3", shape=[3, 3, 16, 16], initializer=xavier_init(16, 16)),
    'conv5_1x1_64' : tf.get_variable(name="conv5_1x1_64", shape=[1, 1, 16, 64], initializer=xavier_init(16, 64)),
    'conv6_1x1' : tf.get_variable(name="conv6_1x1", shape=[1, 1, 64, 16], initializer=xavier_init(64, 16)),
    'conv6_3x3' : tf.get_variable(name="conv6_3x3", shape=[3, 3, 16, 16], initializer=xavier_init(16, 16)),
    'conv6_1x1_64' : tf.get_variable(name="conv6_1x1_64", shape=[1, 1, 16, 64], initializer=xavier_init(16, 64)),
    
    #conv4 지난거 64 + conv6 64 = 128
    
    'dense1' : tf.get_variable(name="dense1", shape=[16*16*128, 1000], initializer=xavier_init(16*16*128, 1000)),
    'dense2' : tf.get_variable(name="dense2", shape=[1000, N_CLASSES], initializer=xavier_init(1000, N_CLASSES)),   
    'logit' : tf.get_variable(name="logit", shape=[16*16*128, N_CLASSES], initializer=xavier_init(16*16*128, N_CLASSES))     
}
#conv net biases 현재 사용하지 않음. 
biases = {
    'conv' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv_b')),
    'conv1_1x1' : tf.Variable(tf.random_normal([4], stddev=stddev, name='conv1_1x1_b')),
    'conv1_3x3' : tf.Variable(tf.random_normal([4], stddev=stddev, name='conv1_3x3_b')),
    'conv1_1x1_16' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv1_1x1_16_b')),
    'conv2_1x1' : tf.Variable(tf.random_normal([4], stddev=stddev, name='conv2_1x1_b')),
    'conv2_3x3' :tf.Variable(tf.random_normal([4], stddev=stddev, name='conv2_3x3_b')),
    'conv2_1x1_16' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv2_1x1_16_b')),
    
    'conv3_1x1' : tf.Variable(tf.random_normal([8], stddev=stddev, name='conv3_1x1_b')),
    'conv3_3x3' : tf.Variable(tf.random_normal([8], stddev=stddev, name='conv3_3x3_b')),
    'conv3_1x1_32' : tf.Variable(tf.random_normal([32], stddev=stddev, name='conv3_1x1_32_b')),
    'conv4_1x1' : tf.Variable(tf.random_normal([8], stddev=stddev, name='conv4_1x1_b')),
    'conv4_3x3' : tf.Variable(tf.random_normal([8], stddev=stddev, name='conv4_3x3_b')),
    'conv4_1x1_32' : tf.Variable(tf.random_normal([32], stddev=stddev, name='conv4_1x1_32_b')),
    
    'conv5_1x1' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv5_1x1_b')),
    'conv5_3x3' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv5_3x3_b')),
    'conv5_1x1_64' : tf.Variable(tf.random_normal([64], stddev=stddev, name='conv5_1x1_64_b')),
    'conv6_1x1' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv6_1x1_b')),
    'conv6_3x3' : tf.Variable(tf.random_normal([16], stddev=stddev, name='conv6_3x3_b')),
    'conv6_1x1_64' : tf.Variable(tf.random_normal([64], stddev=stddev, name='conv6_1x1_64_b')),
    
    'dense1' : tf.Variable(tf.random_normal([1000], stddev=stddev, name='dense1_b')),
    'dense2' : tf.Variable(tf.random_normal([N_CLASSES], stddev=stddev, name='dense2_b'))
}

http://blog.naver.com/PostView.nhn?blogId=skkong89&logNo=220710161978&parentCategoryNo=46&categoryNo=&viewDate=&isShowPopularPosts=false&from=postView
# MODEL

In [ ]:
#model
#http://laonple.blog.me/220764986252 - bottleneck 
#conv 3x3
#conv (1x1, 3x3, 1x1) - relu > 16
#conv (1x1, 3x3, 1x1) relu > 32
#conv (1x1, 3x3, 1x1) relu > 64
#avg pooling 
#fc
#softmax
def ResNet(img_width, img_height, img_channel, _x, _w, _b, scope='ResNet', training=False):
    network = OrderedDict() #network layers

    # X RESHAPE
    _x_r = tf.reshape(_x, shape=[-1,img_width,img_height, img_channel])
    
    with tf.variable_scope(scope):
        with tf.name_scope('conv') as scope:
            conv = tf.nn.conv2d(_x_r, _w['conv'], strides=[1, 1, 1, 1], padding='SAME')
            #conv = tf.nn.bias_add(conv, _b['conv'])
            #conv = batch_norm(conv, conv[0].shape[2], training)#배치노말라이제이션만 해주자 
            #conv = tf.layers.batch_normalization(conv, training=training, name='bn_conv')
            conv = tf.nn.relu(conv)
            network['conv'] = conv
        
        #16
        with tf.name_scope('conv2') as scope:
            conv1_1x1 = tf.nn.conv2d(conv, _w['conv1_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            #conv1_1x1 = tf.nn.bias_add(conv1_1x1, _b['conv1_1x1'])
            conv1_1x1 = tf.nn.relu(conv1_1x1)
            conv1_3x3 = tf.nn.conv2d(conv1_1x1, _w['conv1_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            #conv1_3x3 = tf.nn.bias_add(conv1_3x3, _b['conv1_3x3'])
            conv1_3x3 = tf.nn.relu(conv1_3x3)
            conv1_1x1_16 = tf.nn.conv2d(conv1_3x3, _w['conv1_1x1_16'], strides=[1, 1, 1, 1], padding='SAME')
            #conv1_1x1_16 = tf.nn.bias_add(conv1_1x1_16, _b['conv1_1x1_16'])
            #conv1_1x1_16 = batch_norm(conv1_1x1_16, conv1_1x1_16[0].shape[2], training)
            #conv1_1x1_16 = tf.layers.batch_normalization(conv1_1x1_16, training=training, name='bn_conv1')
            conv1_1x1_16 = tf.nn.relu(conv1_1x1_16)
            network['conv1_1x1_16'] = conv1_1x1_16
        
        #16
        with tf.name_scope('conv3') as scope:
            conv2_1x1 = tf.nn.conv2d(conv1_1x1_16, _w['conv2_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            #conv2_1x1 = tf.nn.bias_add(conv2_1x1, _b['conv2_1x1'])
            conv2_1x1 = tf.nn.relu(conv2_1x1)
            conv2_3x3 = tf.nn.conv2d(conv2_1x1, _w['conv2_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            #conv2_3x3 = tf.nn.bias_add(conv2_3x3, _b['conv2_3x3'])
            conv2_3x3 = tf.nn.relu(conv2_3x3)
            conv2_1x1_16 = tf.nn.conv2d(conv2_3x3, _w['conv2_1x1_16'], strides=[1, 1, 1, 1], padding='SAME')
            #conv2_1x1_16 = tf.nn.bias_add(conv2_1x1_16, _b['conv2_1x1_16'])
            #conv2_1x1_16 = batch_norm(conv2_1x1_16, conv2_1x1_16[0].shape[2], training)
            #conv2_1x1_16 = tf.layers.batch_normalization(conv2_1x1_16, training=training, name='bn_conv2')
            #print(conv2_1x1_16.shape)
            #32
            conv2_1x1_16 = tf.concat([conv, conv2_1x1_16], 3) 
            conv2_1x1_16 = tf.nn.relu(conv2_1x1_16)
            network['conv2_1x1_16'] = conv2_1x1_16
       
        #print(conv2_1x1_16.shape)
        #32       32 x 32 x 16+16 >>> 
        with tf.name_scope('conv4') as scope:
            conv3_1x1 = tf.nn.conv2d(conv2_1x1_16, _w['conv3_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            #conv3_1x1 = tf.nn.bias_add(conv3_1x1, _b['conv3_1x1'])
            conv3_1x1 = tf.nn.relu(conv3_1x1)
            conv3_3x3 = tf.nn.conv2d(conv3_1x1, _w['conv3_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            #conv3_3x3 = tf.nn.bias_add(conv3_3x3, _b['conv3_3x3'])
            conv3_3x3 = tf.nn.relu(conv3_3x3)
            conv3_1x1_32 = tf.nn.conv2d(conv3_3x3, _w['conv3_1x1_32'], strides=[1, 1, 1, 1], padding='SAME')
            #conv3_1x1_32 = tf.nn.bias_add(conv3_1x1_32, _b['conv3_1x1_32'])
            #conv3_1x1_32 = batch_norm(conv3_1x1_32, conv3_1x1_32[0].shape[2], training)
            #conv3_1x1_32 = tf.layers.batch_normalization(conv3_1x1_32, training=training, name='bn_conv3')
            conv3_1x1_32 = tf.nn.relu(conv3_1x1_32)
            network['conv3_1x1_32'] = conv3_1x1_32
        
        with tf.name_scope('conv5') as scope:
            conv4_1x1 = tf.nn.conv2d(conv3_1x1_32, _w['conv4_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            #conv4_1x1 = tf.nn.bias_add(conv4_1x1, _b['conv4_1x1'])
            conv4_1x1 = tf.nn.relu(conv4_1x1)
            conv4_3x3 = tf.nn.conv2d(conv4_1x1, _w['conv4_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            #conv4_3x3 = tf.nn.bias_add(conv4_3x3, _b['conv4_3x3'])
            conv4_3x3 = tf.nn.relu(conv4_3x3)
            conv4_1x1_32 = tf.nn.conv2d(conv4_3x3, _w['conv4_1x1_32'], strides=[1, 1, 1, 1], padding='SAME')
            #conv4_1x1_32 = tf.nn.bias_add(conv4_1x1_32, _b['conv4_1x1_32'])
            #conv4_1x1_32 = batch_norm(conv4_1x1_32, conv4_1x1_32[0].shape[2], training)
            #conv4_1x1_32 = tf.layers.batch_normalization(conv4_1x1_32, training=training, name='bn_conv4')

            #64
            conv4_1x1_32 = tf.concat([conv2_1x1_16, conv4_1x1_32 ], 3)
            conv4_1x1_32 = tf.nn.relu(conv4_1x1_32)
            network['conv4_1x1_32'] = conv4_1x1_32
    
        with tf.name_scope('conv6') as scope:
            conv5_1x1 = tf.nn.conv2d(conv4_1x1_32, _w['conv5_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            #conv5_1x1 = tf.nn.bias_add(conv5_1x1, _b['conv5_1x1'])
            conv5_1x1 = tf.nn.relu(conv5_1x1)
            conv5_3x3 = tf.nn.conv2d(conv5_1x1, _w['conv5_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            #conv5_3x3 = tf.nn.bias_add(conv5_3x3, _b['conv5_3x3'])
            conv5_3x3 = tf.nn.relu(conv5_3x3)
            conv5_1x1_64 = tf.nn.conv2d(conv5_3x3, _w['conv5_1x1_64'], strides=[1, 1, 1, 1], padding='SAME')
            #conv5_1x1_64 = tf.nn.bias_add(conv5_1x1_64, _b['conv5_1x1_64'])
            #conv5_1x1_64 = batch_norm(conv5_1x1_64, conv5_1x1_64[0].shape[2], training)
            #conv5_1x1_64 = tf.layers.batch_normalization(conv5_1x1_64, training=training, name='bn_conv5')
            conv5_1x1_64 = tf.nn.relu(conv5_1x1_64)
            network['conv5_1x1_64'] = conv5_1x1_64
        
        with tf.name_scope('conv7') as scope:
            conv6_1x1 = tf.nn.conv2d(conv5_1x1_64, _w['conv6_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            #conv6_1x1 = tf.nn.bias_add(conv6_1x1, _b['conv6_1x1'])
            conv6_1x1 = tf.nn.relu(conv6_1x1)
            conv6_3x3 = tf.nn.conv2d(conv6_1x1, _w['conv6_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            #conv6_3x3 = tf.nn.bias_add(conv6_3x3, _b['conv6_3x3'])
            conv6_3x3 = tf.nn.relu(conv6_3x3)
            conv6_1x1_64 = tf.nn.conv2d(conv6_3x3, _w['conv6_1x1_64'], strides=[1, 1, 1, 1], padding='SAME')
            #conv6_1x1_64 = tf.nn.bias_add(conv6_1x1_64, _b['conv6_1x1_64'])
            #conv6_1x1_64 = batch_norm(conv6_1x1_64, conv6_1x1_64[0].shape[2], training)
            #conv6_1x1_64 = tf.layers.batch_normalization(conv6_1x1_64, training=training, name='bn_conv6')
            
            #128
            conv6_1x1_64 = tf.concat([conv4_1x1_32, conv6_1x1_64],3)
            conv6_1x1_64 = tf.nn.relu(conv6_1x1_64)
            network['conv6_1x1_64'] = conv6_1x1_64
        
        with tf.name_scope('pool') as scope:
            pool = tf.nn.avg_pool(conv6_1x1_64, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            network['pool'] = pool
        
        with tf.name_scope('dense1') as scope:
            dense = tf.reshape(pool, [-1, _w['dense1'].get_shape().as_list()[0]])
            dense1 = tf.add(tf.matmul(dense, _w['dense1']), _b['dense1'])
            #dense1 = batch_norm(dense1, 1, training)
            dense1 =  tf.layers.batch_normalization(dense1, training=training, name='bn_dense')
            dense1 = tf.nn.relu(dense1)
            network['dense1'] = dense1
            
        #with tf.name_scope('dropout') as scope:
        #    if training:
        #        dropout = tf.nn.dropout(dense1, 0.5, name='dropout')
            
        with tf.name_scope('logit') as scope:
            logit = tf.add(tf.matmul(dense1, _w['dense2']), _b['dense2'])
            network['logit'] = logit
        
    return network

# LOAD DATASET & CHECK LAYERS

In [ ]:
images, labels = distorted_inputs(DATA_PATH, 64)

#labels = tf.cast(labels, tf.float32)
labels = tf.one_hot(indices=labels, depth=10, on_value=1, off_value=0, axis=1)

print(labels.shape)

out = ResNet(INPUT_SIDE, INPUT_SIDE, N_CHANNEL, images, weights, biases, 'ResNet', True)

for key, value in out.items():
    print (key, value)

# LOSS & OPTIMIZER

In [ ]:
#loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=out['logit']))
tf.summary.scalar("loss", loss) #save loss 
print("LOSS FUNCTION")

#learning rate
#global_step = tf.Variable(0, trainable=False, name="global_step")
global_step = 0 #global_step: A scalar int32 or int64 Tensor or a Python number.  
#글로벌 스텝을 텐서로 쓰려면 tf.contrib를 사용해야 할 것 같다.. 
#train_op = tf.contrib.training.create_train_op(total_loss, optimizer, global_step=global_step) 이런느낌? 
learning_rate = tf.train.exponential_decay(learning_rate=LR,
                                           global_step=global_step,
                                           decay_steps=300,
                                           decay_rate=LR_DECAY_RATE,
                                           staircase=True,
                                           name="learning_rate")
learning_rate = tf.maximum(learning_rate, 0.0001)
tf.summary.scalar("learning_rate", learning_rate) #learning rate
print("LERANING RATE : {}".format(learning_rate))

#optimizer
adam = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)
sgd = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)

corr = tf.equal(tf.argmax(out['logit'], 1), tf.argmax(labels,1))
accr = tf.reduce_mean(tf.cast(corr, "float"))
tf.summary.scalar("accuracy", accr) #save accuracy

# SESSION & SAVER & TENSORBOARD 

In [ ]:
#SESSION INITIALIZE
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#SAVER
saver = tf.train.Saver(max_to_keep=3) #최근 3개까지만 저장
save_step = 100 #save for 100 epoch
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
    
#restore checkpoint
checkpoint = tf.train.latest_checkpoint(SAVE_PATH)
if checkpoint is not None:
    print(checkpoint)
    #saver.restore(sess, checkpoint)
    
#TENSOR BOARD
summary_op = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter(BOARD_PATH, sess.graph)

#QUEUE
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=sess)

print("initialized")

# TRAINING

In [ ]:
#training
train_loss_for_plot = []
train_acc_for_plot = []
test_loss_for_plot = []
test_acc_for_plot = []

print('start')
for epoch in range(EPOCHS):
    print(epoch)
    global_stop = epoch #lr decay
    nNumBatch = 0 # batch 수
    AvgBatchCost = 0 #cost 합산
    
    for i in range(int(NUM_BATCHES_PER_EPOCH)):
        nNumBatch += 1
        if epoch < EPOCHS * 0.5:
            _, tmp_cost = sess.run([adam, loss])
        else:
            #print('use sgd')
            _, tmp_cost = sess.run([sgd, loss])
            
        AvgBatchCost += tmp_cost
    
        if nNumBatch % 100 == 0:#print
            train_acc = sess.run(accr)
            print('\t[%d nNumBatch] train cost = %g, acc = %g' %(nNumBatch, AvgBatchCost/nNumBatch, train_acc ))
            lr = sess.run(learning_rate)
            print('\t\t learning rate = %g'%(lr))
            train_loss_for_plot.append(AvgBatchCost/nNumBatch)
            train_acc_for_plot.append(train_acc)
            
    if epoch % 10 == 0: #test
        save_path = saver.save(sess, SAVE_PATH + CHECKPOINT, global_step=epoch)
        print(save_path)

In [ ]:
coord.request_stop()
coord.join(thresds)
print('thread is stopped')
sess.close()
print('session closed')